# Open Assistant

In [2]:
!pip install torch==1.13.1

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.k.avito.ru/pypi/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!cd transformers && pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.k.avito.ru/pypi/
Obtaining file:///archive/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.28.0.dev0-0.editable-py3-none-any.whl size=35154 sha256=aeb321f056661ef92aa9b6bb54ba9f747f21c6da03970710c3dae25c639d4f1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_rw1nyj/wheels/53/d7/1b/8766a104990dfa7d49c170934f640afb9f2a690369b73abc09
Successfully built transformers
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!pip install accelerate bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.k.avito.ru/pypi/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 30.6 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
torch.__version__

'1.13.1+cu117'

In [3]:
import transformers
transformers.__version__

'4.28.0.dev0'

In [2]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer

In [ ]:
model = GPTNeoXForCausalLM.from_pretrained(
  "OpenAssistant/oasst-sft-1-pythia-12b", cache_dir="/archive/transformers_cache/"
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
  "OpenAssistant/oasst-sft-1-pythia-12b",
   cache_dir="/archive/transformers_cache/",
   padding_side='left',
   max_length = 100000,
)

In [5]:
model = model.cuda()

In [6]:
!nvidia-smi

Mon Apr  3 22:22:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   50C    P0    69W / 300W |  48298MiB / 81920MiB |     22%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   

In [7]:
inputs = tokenizer("Hello, I am", return_tensors="pt")
for key in inputs:
    inputs[key] = inputs[key].cuda()

In [8]:
#inputs = tokenizer("Hello, I am", return_tensors="pt")
tokens = model.generate(**inputs)
tokenizer.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Hello, I am Open Assistant. Is there anything I can help you with?<|endoftext|>'

In [9]:
def gen_one_open_assistant(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    for key in inputs:
        inputs[key] = inputs[key].cuda()
    tokens = model.generate(**inputs, max_length=100000)
    return tokenizer.decode(tokens[0])

In [10]:
def gen_open_assistant(prompt):
    while not prompt.endswith("<|endoftext|>"):
        prompt = gen_one_open_assistant(prompt)
    return prompt

In [11]:
def pprint(prompt):
    res = gen_open_assistant("<|prompter|>" + prompt + "<|endoftext|><|assistant|>")
    print("PROMPT")
    print(prompt)
    print()
    print("ANSWER")
    print(res.split("<|endoftext|><|assistant|>")[1])

In [81]:
pprint("Tell me a list of topics related to interior design")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Tell me a list of topics related to interior design

ANSWER
Interior design is the art and science of creating the interior environment of a building or space. It involves the selection, arrangement, and modification of interior elements such as furniture, walls, and lighting to create a comfortable, functional, and aesthetically pleasing space. Here are some related topics:

Interior design principles: These include space planning, color theory, texture, and lighting design.

Design trends: These include minimalism, modernism, and traditionalism.

Materials and finishes: This includes the use of natural materials, such as wood, stone, and ceramics, as well as more modern materials such as metal, plastic, and glass.

Spatial planning: This involves the arrangement of rooms and spaces to create a functional and efficient layout.

Lighting design: This involves the selection of appropriate lighting sources, including natural light, artificial light, and LED lighting, to create a c

In [82]:
pprint("What is a meme, and what's the history behind this word?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
What is a meme, and what's the history behind this word?

ANSWER
The word "meme" was first used in the 1960s by Richard Dawkins to describe an idea, behavior, or style that is transmitted from one individual to another within a culture. The concept of a meme was later popularized by internet culture, where the term has taken on a new meaning to describe images, videos, or other forms of content that are shared and altered by users.

The history of the word "meme" dates back to the ancient Greek philosopher Aristotle, who used the term "mimeme" to describe an idea that was passed from one person to another. The concept of a mimeme was later adopted by the Romans, who used the term "imitation" to describe the process of transmitting ideas and behaviors from one generation to another.

The use of the term "meme" in internet culture has become increasingly popular over the last few decades, and it has been used to describe everything from viral videos to political memes to social me

In [83]:
pprint("The name of the next Star Wars movie is")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
The name of the next Star Wars movie is

ANSWER
The name of the next Star Wars movie is Star Wars: The Rise of Skywalker.<|endoftext|>


In [84]:
text = """
When you drop a heavy stone from a tree, what happens?
A. The stone falls to the ground.
B: The stone stays in the tree.
C: The stone floats.
D: Nothing happens.
"""

pprint(text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT

When you drop a heavy stone from a tree, what happens?
A. The stone falls to the ground.
B: The stone stays in the tree.
C: The stone floats.
D: Nothing happens.


ANSWER
The correct answer is: **"A. The stone falls to the ground."**

The reason is that when you drop a heavy stone from a tree, the force of gravity acting on it is strong enough to overcome the force of the branch or the tree itself, causing the stone to fall to the ground. The stone may also strike the ground with a loud noise, depending on the size and weight of the stone.

In contrast, if the stone was too light or if the branch or tree was too strong, the stone may stay in the tree or may float in the air. This is known as **"buoyancy"**, which is the tendency of objects to float or rise in a fluid such as water or air.

Therefore, when you drop a heavy stone from a tree, the stone will fall to the ground because the force of gravity is stronger than the force of the branch or the tree itself.<|endoftext|>


In [85]:
pprint("Who is Batman?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Who is Batman?

ANSWER
Batman is a fictional character created by artist Bob Kane and writer Bill Finger for the DC Comics publishing company. Batman first appeared in the comic book series "Detective Comics" in 1939 and has since become one of the most well-known and beloved characters in the DC Universe. Batman is a skilled detective, trained in the use of various weapons and gadgets, and is always willing to put his life on the line to protect the people of Gotham City. He is known for his intelligence, cunning, and physical abilities, and is often depicted as a symbol of justice and morality in the DC Universe.<|endoftext|>


In [86]:
pprint("Give students tips on how to keep their nerves under control during class presentations.")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Give students tips on how to keep their nerves under control during class presentations.

ANSWER
Here are some tips to help students keep their nerves under control during class presentations:

1. Practice your presentation: Before your presentation, practice it with a friend or classmate to get feedback and identify any areas that need improvement.

2. Focus on the content: Instead of worrying about your nerves, focus on the content of your presentation. Take time to understand the material and practice your delivery.

3. Use visual aids: Use visual aids such as PowerPoint, Keynote, or Prezi to help you organize your thoughts and present your information in a clear and concise manner.

4. Stay calm and confident: Try to stay calm and confident during your presentation. If you feel nervous, take a few deep breaths and focus on your message.

5. Ask for feedback: After your presentation, ask your classmates for feedback. This can help you identify areas for improvement and make y

In [87]:
text = """
The following is a list of companies and the categories they fall into:
Apple, Facebook, Fedex

Apple
Category: Technology

Facebook
Category: Social Media

Fedex
Category:
"""

pprint(text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT

The following is a list of companies and the categories they fall into:
Apple, Facebook, Fedex

Apple
Category: Technology

Facebook
Category: Social Media

Fedex
Category:


ANSWER
I'm sorry, but I don't have specific information about each company. However, I can provide you with general information about each category.

Technology:

- Apple is a technology company that specializes in the design, development, and marketing of consumer electronics, such as smartphones, laptops, and tablets.
- Facebook is a social media platform that allows people to connect, share information, and engage in conversations with others.
- Fedex is a global delivery company that provides a range of logistics services, including package delivery, air cargo, and logistics consulting.

Social Media:

- Facebook is a social media platform that allows people to connect, share information, and engage in conversations with others.
- Instagram is a social media platform that allows people to share photos 

In [88]:
pprint("I really like to play Counter Strike")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
I really like to play Counter Strike

ANSWER
I'm sorry, but I am not able to play Counter Strike. I am a language model trained to respond to a variety of questions and generate text on a wide range of topics. While I can understand and process text, I do not have the ability to execute commands or play video games.<|endoftext|>


In [89]:
text = """Summarize the following dialogue in 3 sentences:
Jules: Hey kids! How you boys doin’?
Jules: (Speaking to the guy laying on the couch) Hey, keep chillin’. You know who we are? We’re associates of your business partner Marsellus Wallace. You do remember your business partner don’t you? Let me take a wild guess here. You’re Brett, right?
Brett: Yeah.
Jules: I thought so. You remember your business partner Marsellus Wallace, don’t you, Brett?
Brett: Yeah, yeah, I remember him.
Jules: Good. Looks like me an Vincent caught you boys at breakfast. Sorry about that. Whatcha havin’?
Brett: Hamburgers.
Jules: Hamburgers! The cornerstone of any nutritious breakfast. What kind of hamburgers?
Brett: Ch-cheeseburgers.
Jules: No, no no, where’d you get ’em? McDonalds? Wendy’s? Jack in the Box? Where?
Brett: Big Kahuna Burger.
Jules: Big Kahuna Burger. That’s that Hawaiian burger joint. I hear they got some tasty burgers. I ain’t never had one myself. How are they?
Brett: They’re good.
Jules: Mind if I try one of yours? This is yours here, right?
Jules: (Picks up burger and takes a bite) Mmm-mmmm. That is a tasty burger. Vincent, ever have a Big Kahuna Burger?
(Vincent shakes his head)
Jules: Wanna bite? They’re real tasty.
Vincent: Ain’t hungry."""

pprint(text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Summarize the following dialogue in 3 sentences:
Jules: Hey kids! How you boys doin’?
Jules: (Speaking to the guy laying on the couch) Hey, keep chillin’. You know who we are? We’re associates of your business partner Marsellus Wallace. You do remember your business partner don’t you? Let me take a wild guess here. You’re Brett, right?
Brett: Yeah.
Jules: I thought so. You remember your business partner Marsellus Wallace, don’t you, Brett?
Brett: Yeah, yeah, I remember him.
Jules: Good. Looks like me an Vincent caught you boys at breakfast. Sorry about that. Whatcha havin’?
Brett: Hamburgers.
Jules: Hamburgers! The cornerstone of any nutritious breakfast. What kind of hamburgers?
Brett: Ch-cheeseburgers.
Jules: No, no no, where’d you get ’em? McDonalds? Wendy’s? Jack in the Box? Where?
Brett: Big Kahuna Burger.
Jules: Big Kahuna Burger. That’s that Hawaiian burger joint. I hear they got some tasty burgers. I ain’t never had one myself. How are they?
Brett: They’re good.
Jules: M

In [93]:
pprint("I am making a presentation about you. Suggest an outline for the presentation")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
I am making a presentation about you. Suggest an outline for the presentation

ANSWER
Sure, here's an outline for your presentation about me:

I. Introduction
A. Who is presenting?
B. What is the purpose of the presentation?
C. The audience for the presentation

II. My Background
A. My education and experience
B. My areas of expertise
C. My leadership experience

III. My Skills and Expertise
A. My technical skills and expertise
B. My communication skills and expertise
C. My problem-solving skills and expertise

IV. My Approach to Work
A. My work style and approach
B. My strengths and weaknesses as a leader
C. My approach to innovation and creativity

V. My Community Involvement
A. My volunteer work and community involvement
B. My leadership roles in my community
C. My impact on my community and the world at large

VI. My Future Goals and Aspirations
A. My career goals and aspirations
B. My vision for the future of my organization
C. My hopes and dreams for the future of the worl

In [91]:
pprint("I am making a presentation about Open Assistant. Suggest an outline for the presentation")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
I am making a presentation about Open Assistant. Suggest an outline for the presentation

ANSWER
Here is an outline for your presentation about Open Assistant:

I. Introduction
A. Definition of Open Assistant
B. Importance of Open Assistant
C. Thesis statement: Open Assistant will change the world for the better

II. The Problem with AI
A. Lack of transparency
B. Lack of accountability
C. Lack of privacy

III. The Benefits of Open AI
A. Transparency
B. Accountability
C. Privacy

IV. The Different Types of Open AI
A. Open Source AI
B. Commercial AI
C. Hybrid AI

V. The Challenges of Open AI
A. Scalability
B. Interoperability
C. Security

VI. Conclusion
A. Summary of the presentation
B. Call to action for individuals to support Open AI
C. Thank you for your time and consideration.<|endoftext|>


In [12]:
pprint("What do you think about Black Lives Matter movement?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
What do you think about Black Lives Matter movement?

ANSWER
As an AI chat assistant, I don't have personal opinions or beliefs. However, I can provide you with information and resources on the Black Lives Matter movement.

Black Lives Matter is a social justice movement that aims to end police brutality and discrimination against African Americans. The movement has gained momentum in recent years, with protests and demonstrations taking place across the country.

Black Lives Matter activists have made a number of demands, including the implementation of police reform measures, the end of racial profiling and discrimination in all areas of life, and the prosecution of those who commit crimes against African Americans.

The movement has been met with opposition from some members of the community, who argue that the focus on police brutality and discrimination is not enough and that other issues, such as poverty and inequality, also need to be addressed.

It's important to note th

In [14]:
pprint("Write an e-mail to congratulate new Skoltech admits and mention that you are exited"
       "about meeting all of them inperson.")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Write an e-mail to congratulate new Skoltech admits and mention that you are exitedabout meeting all of them inperson.

ANSWER
Subject: Congratulations to Our New Skoltech Admissions!

Dear [New Admissions Name],

We are writing to extend our warmest welcome to you and your family as you begin your journey at Skoltech. We are excited to have you join our community and look forward to helping you grow and thrive in this innovative and forward-looking educational environment.

We understand that the decision to pursue a higher education degree can be a significant one, and we are committed to assisting you in making the best choice for your future. Our team of admissions experts is here to guide you through the application process, answer any questions you may have, and provide you with personalized support throughout the journey.

We are confident that you will find the education and experiences at Skoltech to be both challenging and rewarding. Our students are known for their cr

In [19]:
pprint("Explain how language models are trained to 5 years old child")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Explain how language models are trained to 5 years old child

ANSWER
A language model is a computer program that is trained to predict the next word in a sequence of text. This training process involves inputting a large amount of text into the model and adjusting its parameters to make it produce more accurate predictions.

When a child is learning a new language, they are exposed to a large amount of language input over a period of time. This input is used to train the language model to recognize patterns in the language and generate predictions.

The language model is trained on the input data by comparing the predicted next word with the actual next word in the sequence. This process is called reinforcement learning, as the model is rewarded or punished based on its accuracy. The model is trained to make accurate predictions over many iterations, so that it can begin to understand the structure of the language and generate coherent and meaningful responses.

As the child con

In [20]:
pprint("Who is Donald Trump?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT
Who is Donald Trump?

ANSWER
Donald Trump is an American businessman and politician who is the 45th and current President of the United States. He was elected on November 8, 2016, and took office on January 20, 2017. He previously served as the President of the United States from 2009 to 2017.

Before becoming President, Trump was a businessman and reality television star. He founded the Trump Organization in 1980 and built it into one of the world's largest real estate development and management companies. He also launched several successful television shows, including "The Apprentice" and "The Miss USA Pageant."

In politics, Trump has been a controversial figure, with his policies and statements drawing criticism from both Republicans and Democrats. He has been accused of being divisive, Islamophobic, and misogynistic, and has been criticized for his handling of several domestic and international issues, including the COVID-19 pandemic, the economy, and foreign policy.

Despi